In [6]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os, pyodbc, psycopg2
from mysql import *
from sqlalchemy import create_engine
from dateutil.parser import *
from datetime import *
from dateutil.relativedelta import *
import create_table.tbl_mysql as myq
import create_table.tbl_mssql as msq
from o_fnsql import *
from o_fnsql_2 import *

class OMSQ_B1:
    def __init__(self,srv,user='',password='',host='',db=''):
        self.engin = ''
        self.cn = ''
        if srv.lower == 'oracle':
            self.cn = cx_Oracle.connect(cstr(srv,user,password,host,db))
        else:
            self.engin = create_engine(cstr(srv,user,password,host,db))
            self.cn = self.engin.raw_connection()
        self.T = ''
        self.tbl = ''
        self.C = []
        self.V = []
        self.mdf = pd.DataFrame([])
        self.qrydf = pd.DataFrame([])
        self.tableinfo = {}
        self.dbtype = ''
        self.ins = []
        self.sel_ls = []
        self.upd_ls = []
        self.upd_set = ''
        self.cond = ''
        self.track = defaultdict(dict)
    def S_tblcol(self,tbl,col=False):
        self.T = tbl
        self.C = col
    def S_insert(self,vals=[])
        insert_into = FS_insert(self.T,self.C,vals=[])
        self.ins.append(insert_into)
        self.track = trc_dc(track,'S_insert',insert_into)
        return insert_into
    def S_fault_test(self,vals,condcol,operator=' = '):
        T0 = ' where ' + condcol + operator
        T1 = 'select * from ' + self.T + ' where ' + condcol + operator
        T2 = 'insert into ' + self.T + condcol + ' values '
        cursors = self.conn.cursor()
        for i,v in enumerate(vals):
            if self.C[i] == condcol:
                T0 = T0 + "'" + str(v) + "'"
                T1 = T1 + "'" + str(v) + "'"
                T2 = T2 + "'" + str(v) + "'"
                print('T0-T2 -- ',T0,T1,T2)
                try:
                    cursors.execute(T1)
                    print(T1)
                except:
                    cursors.execute(T2)
                    print(T2)
                finally:
                    break
        else:
            self.conn.commit()
            cursors.close()
        T31 = 'UPDATE ' + self.T + ' SET ' 
        for i,v in enumerate(vals):
            if self.C[i] != condcol:
                T32 = T31 + self.C[i] + "='" + str(v) + "'" + T0
                T42 = T31 + self.C[i] + "='" + str(TM1(v)) + "'" + T0
                try:
                    cursors.execute(T32)
                    print('pass: ', T32)
                except:
                    try:
                        cursors.execute(T42)
                        print('pass: ', T42)
                    except:
                        print('Failed: ', T32, " ~~ ", T42)
    def S_update(self,vals,condcol=[],operator=' LIKE ',condop =' and ',enable_condcol=True):
        self.upd_set = FS_update(vals,condcol,operator,condop,enable_condcol=True)
        self.upd_ls.append(self.upd_set)
        self.track = trc_dc(self.track,'S_update',self.upd_set)
        return self.upd_set
    def S_addcond(self, addby= ' AND ',condition =''):
        if self.cond == '':
            self.cond = ' WHERE ' + condition
        else:
            self.cond = self.cond + addby + condition
        self.track = trc_dc(self.track,'S_addcond',[condition])
    def S_select(self,col=False):
        sel = ''
        if col==False:
            sel = "select " + ','.join(map(str, self.C)) + " from " + self.T
        else:
            if type(col) is list:
                sel = "select " + ','.join(map(str, col)) + " from " + self.T
            else:
                sel = "select " + str(col) + " from " + self.T
        if self.cond !='':
            sel = sel + self.cond
        self.sel_ls.append(sel)
        self.track = trc_dc(self.track,'S_select',sel)
        return sel
    def S_addcond_time(self, T1, T2, db_datetime_col='LASTOCCURRENCE', addby= ' AND '):
        if dbtype == 'oracle':
            if self.cond == '':
                self.cond = ' WHERE ' + timebetween(T1,T2,db_datetime_col)
            else:
                self.cond = self.cond + addby + timebetween(T1,T2,db_datetime_col)
        elif dbtype == 'mssql':
            if self.cond == '':
                self.cond = ' WHERE ' + timebetween(T1,T2,db_datetime_col,dbtype)
            else:
                self.cond = self.cond + addby + timebetween(T1,T2,db_datetime_col,dbtype)
        self.track = trc_dc(self.track,'S_addcond_time',[timebetween(T1,T2,db_datetime_col,dbtype)])
    def G_create_tbl(self,df=False,tbl=False):
        if df == True:
            if self.dbtype == 'mssql':
                msq.CreateTable_MSSQL(df, tbl, self.cn)
            elif self.dbtype == 'mysql':
                myq.CreateTable_MYSQL(connection = self.cn, tablename = tbl, df = df)
        else:
            if self.dbtype == 'mssql':
                msq.CreateTable_MSSQL(self.mdf, self.T, self.cn)
            elif self.dbtype == 'mysql':
                myq.CreateTable_MYSQL(connection = self.cn, tablename = self.T, df = self.mdf)
    def B_sqlgen(self, ndf, bycol=[], tbl = '', sep=','):
        df = rmv_space_in_cols(ndf)
        df = ndf.fillna('$')
        self.mdf = df
        if tbl =='':
            tbl = self.T
        else:
            self.T = tbl
        ls1 = insert_into(df)
        lss1 = ['INSERT INTO ' + tbl + i for i in ls1]
        if bycol is None or bycol==[]:
            df1 = pd.DataFrame(ls1, columns=['INS'])
            df1['INS'] = df1.apply(lambda x : 'INSERT INTO ' + tbl + x['INS'], axis = 1)
            self.qrydf = df1
        else:
            ls2 = update_into(df,bycols=bycol)
            dfn = df[bycol]
            ls3 = update_into(dfn,bycols=[],seperator=sep,operator='=')
            lss2 = ['UPDATE ' + tbl + ' SET ' + i + ' WHERE ' + j for i, j in zip(ls2, ls3)]
            self.qrydf = pd.DataFrame(zip(lss2,lss1),columns=['UPD','INS'])
        print('B_sqlgen -- done')
        return self.qrydf
    def B_export(self, ifexist='fail'):
        try:
            self.mdf.to_sql(self.T, con=self.engi, if_exists=ifexist, chunksize=10000)
            print('try success')
        except:
            self.G_create_tbl()
            if self.dbtype == 'mssql':
                self.pyodbc_mssql()
            cr = self.cn.cursor()
            for i in range(len(self.qrydf)):
                for j in self.qrydf:
                    q = self.qrydf.loc[i,j]
                    x = q.find(" WHERE ")
                    if x != -1:
                        tb = q.find(self.tbl)
                        qq = 'select 1 from ' + self.T + ' ' + q[x:len(q)]
                        ddf = pd.read_sql(qq,con=self.conn)
                        if ddf.shape[0]>0:
                            try:
                                cr.execute(q)
                                break
                            except:
                                print('failed: ', q)
                    else:
                        try:
                            cr.execute(q)
                            break
                        except:
                            print('failed: ', q)
                if (i+1)%500==0:
                    self.cn.commit()
                    print(i)
            else:
                self.cn.commit()
                cr.close()
                print('over')
    def execommit(self,q):
        cr = self.cn.cursor()
        rs = cr.execute(q)
        self.cn.commit()
        cr.close
    def exe_get_rs(self, qry):
        cr = self.cn.cursor()
        rs = cr.execute(qry)
        return rs
    def exe_get_df(self, qry):
        df = pd.read_sql(qry, con= self.engin)
        return df
    def table_info(self):
        if self.dbtype == 'MSSQL':
            self.tableinfo = mssql_table_colinfo(self.T,self.cn)
        elif self.dbtype == 'MYSQL':
            self.tableinfo = mysql_table_colinfo(self.D, self.T, self.cn)
        else:
            print('No DB initiated yet')
    def tss(self,x):
        print('omi',x)
    @property
    def ex(self):
        self.tss()

def csv2df(csvfl): 
    ndf = pd.read_csv(csv1)
    xdf = mod_datetime(ndf)
    df = rmv_space_in_cols(xdf)
    return df

csv1 = "D:\\omEngin\\AOmPy_o_\\csv_o_\\sclick3.csv"
ndf = csv2df(csv1)
col = ndf.columns.to_list()
val = ndf.values.tolist()
x = OMSQ_B1(user='sa',password='1q2w3eaz$',host='192.168.0.102',db='omdb')
#x = OMSQ_B1(user='root',password='admin',host='127.0.0.1:3306',db='odb')
x.dbconn('postg')
x.S_tblcol('tbl26',col)
#x.S_addcond(condition="TYPE=1")
#x.S_addcond(addby=' OR ',condition="CUSTOMERATTR15 LIKE 'PBSDR01'")
x.B_sqlgen(ndf,['Serial'])
x.B_export()
#x.S_select(col = ['dfgdf','dfgdff'])
#x.S_select()
#x.S_update(val[5],['Serial','CustomAttr11'],operator=' = ',condop =' and ')

SyntaxError: invalid syntax (<ipython-input-6-8f875947d614>, line 41)

In [ ]:
class OMSQ_M1(OMSQ_B1):
    def __init__(self, idf = pd.DataFrame([])):
        self.df = idf
    def set_conn(self, user='',password='',host='',dbname=''):
        self.U = user
        self.P = password
        self.H = host
        self.D = dbname
    def set_tbl_cols(self,tbl,cols=False):
        self.T = tbl
        self.C = cols
        self.table_info()
    def oracle_default(self):
        self.conn = cx_Oracle.connect('SOC_READ','soc_read', 'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        self.dbtype = 'oracle'
        print(self.conn)
    def oracle_custom(self):
        self.conn = cx_Oracle.connect(self.U,self.P,'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        self.dbtype = 'oracle'
        print(self.conn)
    def mssql(self):
        cstr2 = "mssql+pyodbc://" + self.U + ':' + self.P + '@' + self.H + "/" + self.D + "?driver=SQL Server"
        self.engin = create_engine(cstr2)
        self.cn = self.engin.raw_connection()
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def pyodbc_mssql(self):
        cstr = "Driver={SQL Server};SERVER=" + self.H + ";DATABASE=" + self.D + ";UID=" + self.U + ";PWD=" + self.P
        self.cn = pyodbc.connect(cstr)
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def mysql(self):
        constr = 'mysql+mysqlconnector://' + self.U + ':' + self.P + '@' + self.H + '/' + self.D
        self.engin = create_engine(constr, echo=False)
        self.cn = self.conn.raw_connection()
        self.dbtype = 'MYSQL'
        print('conn succ mysql')
    def ex(self, qry):
        self.qdf = pd.read_sql(qry, con = self.engin)
        return qdf
    def get_conn_oracle(self):
        return self.conn
    def get_conn_other(self):
        return self.cn
    def set_df(self,ndf):
        self.df = ndf
    def check_table(self, tbl):
        return is_table_exist(self.conn,self.table)
    def get_tableinfo(self):
        return self.tableinfo
    def get_qrydf(self):
        return self.qrydf
    def cursor_execute(self, qry):
        cr = self.cn.cursor()
        rs = cr.execute(qry)
        return rs
    def engin_execute(self, qry):
        df = pd.read_sql(qry, con= self.engin)
        return df
    
